In [1]:
from ase import io

import ase

In [2]:
# traj = io.read("OUTCAR")

# from ase.io.vasp import read_vasp_out

# read_vasp_out(filename="OUTCAR", index=-1, force_consistent=False)

In [3]:
filename = "OUTCAR"
index  = -1
force_consistent=False

In [4]:
import os
import re
import ase.units

from ase.utils import basestring

In [5]:
import numpy as np
from ase.calculators.singlepoint import SinglePointCalculator
from ase import Atoms, Atom

try:  # try to read constraints, first from CONTCAR, then from POSCAR
    constr = read_vasp('CONTCAR').constraints
except Exception:
    try:
        constr = read_vasp('POSCAR').constraints
    except Exception:
        constr = None

if isinstance(filename, basestring):
    f = open(filename)
else:  # Assume it's a file-like object
    f = filename
data = f.readlines()
natoms = 0
images = []
atoms = Atoms(pbc=True, constraint=constr)
energy = 0
species = []
species_num = []
stress = None
symbols = []
ecount = 0
poscount = 0
magnetization = []
magmom = None

for n, line in enumerate(data):
    if re.search('[0-9]-[0-9]', line):
#         print(line)
        tmp = re.sub('([0-9])-([0-9])',r'\1 -\2', line)
#         print(tmp)
#         print("")
        data[n] = tmp

In [6]:
for n, line in enumerate(data):
    if 'POTCAR:' in line:
        temp = line.split()[2]
        for c in ['.', '_', '1']:
            if c in temp:
                temp = temp[0:temp.find(c)]
        species += [temp]
    if 'ions per type' in line:
        species = species[:len(species) // 2]
        temp = line.split()
        ntypes = min(len(temp)-4, len(species))
        for ispecies in range(ntypes):
            species_num += [int(temp[ispecies + 4])]
            natoms += species_num[-1]
            for iatom in range(species_num[-1]):
                symbols += [species[ispecies]]
    if 'direct lattice vectors' in line:
        cell = []
        for i in range(3):
            temp = data[n + 1 + i].split()
            cell += [[float(temp[0]), float(temp[1]), float(temp[2])]]
        atoms.set_cell(cell)
    if 'FREE ENERGIE OF THE ION-ELECTRON SYSTEM' in line:
        # choose between energy wigh smearing extrapolated to zero
        # or free energy (latter is consistent with forces)
        energy_zero = float(data[n + 4].split()[6])
        energy_free = float(data[n + 2].split()[4])
        energy = energy_zero
        if force_consistent:
            energy = energy_free
        if ecount < poscount:
            # reset energy for LAST set of atoms, not current one -
            # VASP 5.11? and up
            images[-1].calc.results['energy'] = energy
            images[-1].calc.set(energy=energy)
        ecount += 1
    if 'magnetization (x)' in line:
        magnetization = []
        print("magnetization line num:", n)
        for i in range(natoms):
            magnetization += [float(data[n + 4 + i].split()[4])]
    if 'number of electron' in line:
        parts = line.split()
        if len(parts) > 5 and parts[0].strip() != "NELECT":
            magmom = float(parts[5])
    if 'in kB ' in line:
        stress = -np.array([float(a) for a in line.split()[2:]])
        stress = stress[[0, 1, 2, 4, 5, 3]] * 1e-1 * ase.units.GPa
    if 'POSITION          ' in line:
        print("line num:", n)

        forces = []
        positions = []
        for iatom in range(natoms):
            temp = data[n + 2 + iatom].split()
            atoms += Atom(symbols[iatom],
                          [float(temp[0]), float(temp[1]), float(temp[2])])
            try:
                forces += [[float(temp[3]), float(temp[4]), float(temp[5])]]
            except:
                forces += [[0., 0., 0.]]

            positions += [[float(temp[0]), float(temp[1]), float(temp[2])]]
        atoms.set_calculator(SinglePointCalculator(atoms,
                                                   energy=energy,
                                                   forces=forces,
                                                   stress=stress))
        images += [atoms]
        if len(magnetization) > 0:
            mag = np.array(magnetization, float)
            images[-1].calc.magmoms = mag
            images[-1].calc.results['magmoms'] = mag
        if magmom is not None:
            images[-1].calc.results['magmom'] = magmom
        atoms = Atoms(pbc=True, constraint=constr)
        poscount += 1

magnetization line num: 3352
line num: 3550
magnetization line num: 4811
line num: 5009
magnetization line num: 6223
line num: 6421
magnetization line num: 7745
line num: 7943
magnetization line num: 9157
line num: 9355
magnetization line num: 10467
line num: 10665
magnetization line num: 11777
line num: 11975
magnetization line num: 13181
line num: 13379
magnetization line num: 14644
line num: 14842
magnetization line num: 15947


ValueError: could not convert string to float: 'time'

In [ ]:
# return requested images, code borrowed from ase/io/trajectory.py
if isinstance(index, int):
    return images[index]
else:
    step = index.step or 1
    if step > 0:
        start = index.start or 0
        if start < 0:
            start += len(images)
        stop = index.stop or len(images)
        if stop < 0:
            stop += len(images)
    else:
        if index.start is None:
            start = len(images) - 1
        else:
            start = index.start
            if start < 0:
                start += len(images)
        if index.stop is None:
            stop = -1
        else:
            stop = index.stop
            if stop < 0:
                stop += len(images)
    return [images[i] for i in range(start, stop, step)]